## Coleta de dados

Esse projeto em PySpark faz uma análise de dados de preços de combustíveis da ANP.

O trabalho faz uso de dados publicados pela ANP (Agência Nacional de Petróleo)

> Em cumprimento às determinações da Lei do Petróleo (Lei nº 9478/1997, artigo 8º), a ANP acompanha os preços praticados por revendedores de combustíveis automotivos e de gás liquefeito de petróleo envasilhado em botijões de 13 quilos (GLP P13), por meio de uma pesquisa semanal de preços realizada por empresa contratada.

- [Série Histórica de Preços de Combustíveis e de GLP](https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/serie-historica-de-precos-de-combustiveis)

Coletamos a série histórica de "Combustíveis automotivos" que vai de 2004 a 2023. São 39 arquivos CSV totalizando aproximadamente 3.7 GB.

![Amostra dos dados](./assets/amostra_planilha_revenda.png)

- [Metadados em PDF](https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/metadados-serie-historica-precos-combustiveis-1.pdf)

Temos as seguintes colunas nos arquivos CSV, em conformidade com a documentação de metadados:

| Coluna            | Tipo    | Comentário                                                                           |
| ----------------- | ------- | ------------------------------------------------------------------------------------ |
| Regiao            | Texto   | Sigla da região do país (ex: S, N, SE)                                               |
| Estado            | Texto   | Sigla de unidade federativa (ex: RJ, SP, MG)                                         |
| Municipio         | Texto   | Nome de município                                                                    |
| Revenda           | Texto   | Razão social do revendedor de combustível                                            |
| CNPJ da Revenda   | Texto   | CNPJ do revendedor                                                                   |
| Nome da Rua       | Texto   | Logradouro                                                                           |
| Numero Rua        | Texto   | Logradouro                                                                           |
| Complemento       | Texto   | Logradouro                                                                           |
| Bairro            | Texto   | Logradouro                                                                           |
| CEP               | Texto   | Código de Endereçamento Postal                                                       |
| Produto           | Texto   | Produto combustível (ex: GASOLINA, ETANOL, DIESEL)                                   |
| Data da Coleta    | Data    | Data da pesquisa de preço (formato dd/mm/aaaa)                                       |
| Valor de Venda    | Decimal | Valor de venda da unidade de combustível (4 casas decimais, vírgula como separador)  |
| Valor de Compra   | Decimal | Valor de compra da unidade de combustível (4 casas decimais, virgula como separador) |
| Unidade de Medida | Texto   | Unidade ao qual os valores de compra e venda se referem (ex: R$ / litro)             |
| Bandeira          | Texto   | Nome de marca do posto de revenda (ex: IPIRANGA, BRANCA, COSAN, etc.)                |


## Infraestrutura

Aproveitando a configuração de cluster da Google Cloud Platform criada para o [projeto Hadoop](https://github.com/pinei/hadoop-precos-combustiveis), incluímos o JupyterLab para trabalhar com o PySpark.

![cluster-info](./assets/cluster_info.png)

O suporte ao Hive precisa de um bucket para funcionar em modo distribuído. O caminho para o bucket pode ser especificado em uma propriedade do cluster, antes da sua criação, conforme exemplo:

```
spark:spark.sql.warehouse.dir=gs://hadoop-stage/spark-warehouse
```

Em paralelo usamos uma instalação em Raspberry Pi 4 para verificar a viabilidade e desempenho nesse tipo de hardware de baixo custo.

- [Running PySpark in JupyterLab on a Raspberry Pi](https://dev.to/pinei/running-pyspark-in-jupyterlab-on-a-raspberry-pi-1293)

Por rodar no modo standalone, vamos chamar este ambiente de "máquina local".

Em projeto anterior os arquivos CSV foram organizados em um bucket do Google Cloud Storage.

> `https://console.cloud.google.com/storage/browser/hadoop-dados-brutos`

No cluster Dataproc o Spark tem acesso padrão aos arquivos desse storage via URL `gs://hadoop-dados-brutos`, podendo inclusive fazer o processamento distribuído entre os nós.

In [1]:
DATA_PATH = 'gs://hadoop-dados-brutos'

No Spark em máquina local, o suporte a conexão com o mesmo storage precisa de configuração adicional.

- [How to fix "No FileSystem for scheme: gs" in pyspark?](https://stackoverflow.com/questions/55595263/how-to-fix-no-filesystem-for-scheme-gs-in-pyspark)

## Iniciando a sessão do Spark

Inicializamos uma sessão do Spark com suporte ao Hive para gerenciamento de metadados das bases de dados.

Também habilitamos o suporte ao sistema de arquivos do Google Cloud.

In [2]:
from pyspark.sql import SparkSession

spark = ( SparkSession
    .builder
    .appName("analise-dados")
    .config("spark.jars", "https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-latest.jar")
    .enableHiveSupport()
    .getOrCreate() )

spark._jsc.hadoopConfiguration().set('fs.gs.impl', 'com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem')

23/10/13 04:33:08 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Quando executando fora da Google Cloud, o conector do Google precisa de uma chave para autenticação no serviço.

Passo a passo para gerar a chave:
- Console do Google Cloud Platform
- APIs e Serviços > Credenciais
- Criar Conta de Serviço
- Criar Chave do tipo JSON

> "É feito o download de um arquivo contendo a chave privada. Armazene o arquivo com segurança porque essa chave não pode ser recuperada em caso de perda."

Subimos o arquivo JSON para uma pasta de trabalho:

> `/home/aldinei_bastos/work/seventh-abacus-395221-52fc140a5609.json`

É necessário então localizar o JSON através de uma variável de ambiente `GOOGLE_APPLICATION_CREDENTIALS`

In [3]:
import os

HOME_DIR = os.environ["HOME"]
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = f'{HOME_DIR}/work/seventh-abacus-395221-52fc140a5609.json'

In [ ]:
spark.catalog.listDatabases()

In [ ]:
def create_database(name, comment):
    spark.sql(f"CREATE DATABASE IF NOT EXISTS {name} COMMENT '{comment}'")
    return spark.sql(f"DESCRIBE DATABASE {name}")

metadata = create_database("precos_anp", "Base de dados de preços de combustíveis fornecidos pela ANP e alguns indicadores de mercado")
metadata.show()

## Ingestão

Definimos o schema para leitura dos arquivos CSV.

In [4]:
from pyspark.sql.types import StructType, StructField, StringType, DateType, DecimalType

schema = StructType([
    StructField("Regiao", StringType(), True),
    StructField("Estado", StringType(), True),
    StructField("Municipio", StringType(), True),
    StructField("Revenda", StringType(), True),
    StructField("CNPJ da Revenda", StringType(), True),
    StructField("Nome da Rua", StringType(), True),
    StructField("Numero Rua", StringType(), True),
    StructField("Complemento", StringType(), True),
    StructField("Bairro", StringType(), True),
    StructField("CEP", StringType(), True),
    StructField("Produto", StringType(), True),
    StructField("Data da Coleta", DateType(), True),
    StructField("Valor de Venda", DecimalType(10, 4), True),
    StructField("Valor de Compra", DecimalType(10, 4), True),
    StructField("Unidade de Medida", StringType(), True),
    StructField("Bandeira", StringType(), True)
])

df_csv = spark.read.csv(f"{DATA_PATH}/anp-combustiveis-automotivos/ca-*.csv",
                    header=True,
                    schema=schema,
                    sep=';',
                    multiLine=True,
                    quote='"',
                    dateFormat="dd/MM/yyyy",
                    locale='pt-BR')
df_csv.count()

22645940

Criamos uma tabela gerenciada e populamos com os dados do dataframe carregado

In [ ]:
spark.sql("""
    CREATE TABLE IF NOT EXISTS precos_anp.combustiveis_automotivos (
      regiao STRING,
      estado STRING,
      municipio STRING,
      revenda STRING,
      cnpj_revenda STRING,
      endereco_rua STRING,
      endereco_numero STRING,
      endereco_complemento STRING,
      endereco_bairro STRING,
      endereco_cep STRING,
      mes STRING,
      data DATE,
      valor_venda DECIMAL(10,4),  -- Tipo decimal com até 4 casas decimais
      valor_compra DECIMAL(10,4),  -- Tipo decimal com até 4 casas decimais
      unidade_medida STRING,
      bandeira STRING
    )
    PARTITIONED BY (produto STRING)
    CLUSTERED BY (mes) INTO 8 BUCKETS
""")

# Checando
spark.table('precos_anp.combustiveis_automotivos')

In [5]:
from pyspark.sql import functions as F

df_ca = (
    df_csv.select(
        F.column("Regiao").alias("regiao"),
        F.column("Estado").alias("estado"),
        F.column("Municipio").alias("municipio"),
        F.column("Revenda").alias("revenda"),
        F.column("CNPJ da Revenda").alias("cnpj_revenda"),
        F.column("Nome da Rua").alias("endereco_rua"),
        F.column("Numero Rua").alias("endereco_numero"),
        F.column("Complemento").alias("endereco_complemento"),
        F.column("Bairro").alias("endereco_bairro"),
        F.column("CEP").alias("endereco_cep"),
        F.column("Produto").alias("produto"),
        F.column("Data da Coleta").alias("data"),
        F.column("Valor de Venda").alias("valor_venda"),
        F.column("Valor de Compra").alias("valor_compra"),
        F.column("Unidade de Medida").alias("unidade_medida"),
        F.column("Bandeira").alias("bandeira")
    )
    .withColumn("mes", F.date_format("data", "yyyy-MM"))
    .filter(F.col("produto").isin('DIESEL S10', 'DIESEL', 'ETANOL', 'GNV', 'GASOLINA', 'DIESEL S50', 'GASOLINA ADITIVADA'))
)

df_ca.write.mode("overwrite").saveAsTable("precos_anp.combustiveis_automotivos")

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used
23/10/13 04:36:28 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


In [6]:
df_grouped = spark.table("precos_anp.combustiveis_automotivos").groupBy("produto").count()

df_grouped.show()

+------------------+-------+
|           produto|  count|
+------------------+-------+
|        DIESEL S10|1978185|
|            DIESEL|5406872|
|            ETANOL|6896581|
|               GNV| 435900|
|          GASOLINA|7457216|
|GASOLINA ADITIVADA| 426690|
|        DIESEL S50|  44495|
+------------------+-------+



Como segunda tabela de dados, coletamos a série histórica mensal dos preços de distribuição de combustíveis líquidos, com dados desde 2020/09.

![Dados de distribuição](./assets/amostra_planilha_distribuicao.png)

O arquivo está disponível no bucket com o nome `combustiveis-liquidos-municipios.csv`.

In [7]:
from pyspark.sql.types import StructType, StructField, StringType, DateType, DecimalType

schema = StructType([
    StructField("MÊS", StringType(), True),
    StructField("PRODUTO", StringType(), True),
    StructField("REGIÃO", StringType(), True),
    StructField("ESTADO", StringType(), True),
    StructField("MUNICÍPIO", StringType(), True),
    StructField("UNIDADE", StringType(), True),
    StructField("PREÇO MÉDIO DE DISTRIBUIÇÃO", DecimalType(10, 4), True),
    StructField("DESVIO PADRÃO", DecimalType(10, 4), True)
])

df_csv = spark.read.csv(f"{DATA_PATH}/anp-combustiveis-automotivos/combustiveis-liquidos-municipios.csv",
                    header=True,
                    schema=schema,
                    sep=';',
                    multiLine=True,
                    quote='"',
                    dateFormat="dd/MM/yyyy",
                    locale='pt-BR')
df_csv.count()

80327

In [8]:
spark.sql("""
    CREATE TABLE IF NOT EXISTS precos_anp.distribuicao_combustiveis (
      mes STRING,
      produto STRING,
      regiao STRING,
      estado STRING,
      municipio STRING,
      unidade_medida STRING,
      preco_medio DECIMAL(10,4),
      desvio_padrao DECIMAL(10,4)
    )
""")

# Checando
spark.table('precos_anp.distribuicao_combustiveis')

23/10/13 04:44:36 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.


DataFrame[mes: string, produto: string, regiao: string, estado: string, municipio: string, unidade_medida: string, preco_medio: decimal(10,4), desvio_padrao: decimal(10,4)]

In [13]:
from pyspark.sql import functions as F

df_dc = (
    df_csv.select(
        F.column("MÊS").alias("mes"),
        F.column("PRODUTO").alias("produto"),
        F.column("REGIÃO").alias("regiao"),
        F.column("ESTADO").alias("estado"),
        F.column("MUNICÍPIO").alias("municipio"),
        F.column("UNIDADE").alias("unidade_medida"),
        F.column("PREÇO MÉDIO DE DISTRIBUIÇÃO").alias("preco_medio"),
        F.column("DESVIO PADRÃO").alias("desvio_padrao")
    )
    .withColumn("produto",
        F.when(F.col("produto") == 'ETANOL HIDRATADO COMUM', 'ETANOL')
         .when(F.col("produto") == 'GASOLINA C COMUM', 'GASOLINA')
         .when(F.col("produto") == 'GASOLINA C COMUM ADITIVADA', 'GASOLINA ADITIVADA')
         .when(F.col("produto") == 'ÓLEO DIESEL B S10 - COMUM', 'DIESEL S10')
         .when(F.col("produto") == 'ÓLEO DIESEL B S500 - COMUM', 'DIESEL S500')
         .otherwise(None).alias('produto'))
    .withColumn("regiao",
        F.when(F.col("regiao") == 'CENTRO OESTE', 'CO')
         .when(F.col("regiao") == 'NORDESTE', 'NE')
         .when(F.col("regiao") == 'NORTE', 'N')
         .when(F.col("regiao") == 'SUDESTE', 'SE')
         .when(F.col("regiao") == 'SUL', 'S')
         .otherwise(None).alias('regiao'))
    .withColumn("estado",
        F.when(F.col("estado") == 'ACRE', 'AC')
         .when(F.col("estado") == 'ALAGOAS', 'AL')
         .when(F.col("estado") == 'AMAZONAS', 'AM')
         .when(F.col("estado") == 'AMAPA', 'AP')
         .when(F.col("estado") == 'BAHIA', 'BA')
         .when(F.col("estado") == 'CEARA', 'CE')
         .when(F.col("estado") == 'DISTRITO FEDERAL', 'DF')
         .when(F.col("estado") == 'ESPIRITO SANTO', 'ES')
         .when(F.col("estado") == 'GOIAS', 'GO')
         .when(F.col("estado") == 'MARANHAO', 'MA')
         .when(F.col("estado") == 'MINAS GERAIS', 'MG')
         .when(F.col("estado") == 'MATO GROSSO DO SUL', 'MS')
         .when(F.col("estado") == 'MATO GROSSO', 'MT')
         .when(F.col("estado") == 'PARA', 'PA')
         .when(F.col("estado") == 'PARAIBA', 'PB')
         .when(F.col("estado") == 'PERNAMBUCO', 'PE')
         .when(F.col("estado") == 'PIAUI', 'PI')
         .when(F.col("estado") == 'PARANA', 'PR')
         .when(F.col("estado") == 'RIO DE JANEIRO', 'RJ')
         .when(F.col("estado") == 'RIO GRANDE DO NORTE', 'RN')
         .when(F.col("estado") == 'RONDONIA', 'RO')
         .when(F.col("estado") == 'RORAIMA', 'RR')
         .when(F.col("estado") == 'RIO GRANDE DO SUL', 'RS')
         .when(F.col("estado") == 'SANTA CATARINA', 'SC')
         .when(F.col("estado") == 'SERGIPE', 'SE')
         .when(F.col("estado") == 'SAO PAULO', 'SP')
         .when(F.col("estado") == 'TOCANTINS', 'TO')
         .otherwise(None).alias('estado'))
    .filter((F.trim(F.col("preco_medio")) != '') & (F.trim(F.col("preco_medio")) != '-'))
)

df_dc.write.mode("overwrite").saveAsTable("precos_anp.distribuicao_combustiveis")

In [14]:
df_grouped = spark.table("precos_anp.distribuicao_combustiveis").groupBy("produto").count()

df_grouped.show()

+------------------+-----+
|           produto|count|
+------------------+-----+
|        DIESEL S10|16062|
|GASOLINA ADITIVADA|16056|
|       DIESEL S500|15246|
|            ETANOL|15892|
|          GASOLINA|16065|
+------------------+-----+



Estamos incluindo na análise os preços do petróleo no mercado mundial

- [West Texas Intermediate (WTI)](https://fred.stlouisfed.org/series/DCOILWTICO) de 2003 a 2023
- [Crude Oil Prices: Brent - Europe](https://fred.stlouisfed.org/series/DCOILBRENTEU) de 2003 a 2023

## Analítico

Algumas perguntas para análise dos dados com consultas SQL de exemplo

### 1. Qual a distribuição de CNPJs por bandeira?

In [15]:
from pyspark.sql import functions as F

df_result = (
    spark.table('precos_anp.combustiveis_automotivos')
    .groupBy("bandeira")
    .agg(F.countDistinct("cnpj_revenda").alias("contagem"))
    .orderBy(F.desc("contagem"))
)

df_result.show()

+--------------------+--------+
|            bandeira|contagem|
+--------------------+--------+
|              BRANCA|   26021|
|PETROBRAS DISTRIB...|    9208|
|            IPIRANGA|    7126|
|              RAIZEN|    5819|
|                CBPI|    4033|
|       VIBRA ENERGIA|    3549|
| COSAN LUBRIFICANTES|    2395|
|              ALESAT|    1774|
|            LIQUIGÁS|     498|
|               SABBÁ|     484|
|            SATELITE|     463|
|    ALE COMBUSTÍVEIS|     428|
|                  SP|     415|
|                DPPI|     400|
|             ATEM' S|     232|
|             EQUADOR|     229|
|            PETROSUL|     214|
|               ASTER|     214|
|   TDC DISTRIBUIDORA|     204|
|              DISLUB|     187|
+--------------------+--------+
only showing top 20 rows



### 2. Qual o preço médio de revenda de combustíveis por mês, produto, estado?

Consideramos declarar criar uma tabela intermediária para facilitar o reuso em consultas subsequentes.

In [16]:
from pyspark.sql import functions as F

df_result = (
    spark.table('precos_anp.combustiveis_automotivos')
    .groupBy("mes", "regiao", "estado", "produto")
    .agg(F.avg("valor_venda").alias("valor_venda_medio"))
    .orderBy("mes", "regiao", "estado", "produto", ascending=[False, True, True, True])
)

df_result.write.mode("overwrite").saveAsTable("precos_anp.media_venda_combustiveis_automotivos")

+-------+------+------+------------------+-----------------+
|    mes|regiao|estado|           produto|valor_venda_medio|
+-------+------+------+------------------+-----------------+
|2023-06|    CO|    DF|            DIESEL|       5.01061728|
|2023-06|    CO|    DF|        DIESEL S10|       5.18187500|
|2023-06|    CO|    DF|            ETANOL|       3.83377049|
|2023-06|    CO|    DF|          GASOLINA|       5.42500000|
|2023-06|    CO|    DF|GASOLINA ADITIVADA|       5.50687151|
|2023-06|    CO|    DF|               GNV|       6.04000000|
|2023-06|    CO|    GO|            DIESEL|       4.90575058|
|2023-06|    CO|    GO|        DIESEL S10|       5.02353933|
|2023-06|    CO|    GO|            ETANOL|       3.75582880|
|2023-06|    CO|    GO|          GASOLINA|       5.35765730|
|2023-06|    CO|    GO|GASOLINA ADITIVADA|       5.48975000|
|2023-06|    CO|    MS|            DIESEL|       5.16706250|
|2023-06|    CO|    MS|        DIESEL S10|       5.22741758|
|2023-06|    CO|    MS| 

Fazendo uma comparação do preço médio da gasolina entre os estados, considerando o último mês da série (2023-06)

In [18]:
from pyspark.sql import functions as F

df_result = (
    spark.table('precos_anp.media_venda_combustiveis_automotivos')
    .select("regiao", "estado", "valor_venda_medio")
    .filter((F.col("mes") == '2023-06') & (F.col("produto") == 'GASOLINA'))
    .orderBy("valor_venda_medio", ascending=False)
)

df_result.show()

+------+------+-----------------+
|regiao|estado|valor_venda_medio|
+------+------+-----------------+
|     N|    AM|       6.19634855|
|     N|    AC|       6.09985915|
|     N|    RO|       5.91151899|
|     N|    RR|       5.74308824|
|    NE|    RN|       5.70969849|
|     N|    TO|       5.61351351|
|    SE|    ES|       5.58682464|
|    NE|    BA|       5.57350569|
|    NE|    AL|       5.55412556|
|     S|    SC|       5.54860000|
|     N|    PA|       5.48006865|
|     S|    PR|       5.44663198|
|    NE|    CE|       5.43531457|
|     S|    RS|       5.42974790|
|    CO|    DF|       5.42500000|
|    NE|    PE|       5.42040945|
|    CO|    MS|       5.41880597|
|    SE|    RJ|       5.41836054|
|    NE|    SE|       5.41771242|
|    CO|    MT|       5.39057692|
+------+------+-----------------+
only showing top 20 rows



Os dados indicam que os valores mais altos se concentram no norte do país.

### 3. Qual o preço médio de distribuição de combustíveis por mês, produto, estado?

Consideramos declarar uma tabela intermediária para facilitar o reuso em consultas subsequentes.

In [21]:
from pyspark.sql import functions as F

df_result = (
    spark.table('precos_anp.distribuicao_combustiveis')
    .groupBy("mes", "regiao", "estado", "produto")
    .agg(F.avg("preco_medio").alias("valor_distribuicao_medio"))
    .orderBy("mes", "regiao", "estado", "produto", ascending=[False, True, True, True])
)

df_result.write.mode("overwrite").saveAsTable("precos_anp.media_distribuicao_combustiveis_automotivos")

Fazendo uma comparação do preço médio de distribuição da gasolina entre os estados, considerando o último mês da série (2023-06)

In [22]:
from pyspark.sql import functions as F

df_result = (
    spark.table('precos_anp.media_distribuicao_combustiveis_automotivos')
    .select("regiao", "estado", "valor_distribuicao_medio")
    .filter((F.col("mes") == '2023-06') & (F.col("produto") == 'GASOLINA'))
    .orderBy("valor_distribuicao_medio", ascending=False)
)

df_result.show()

+------+------+------------------------+
|regiao|estado|valor_distribuicao_medio|
+------+------+------------------------+
|     N|    AC|              5.03100000|
|     N|    RR|              4.90800000|
|    SE|    ES|              4.87740000|
|    NE|    AL|              4.87720000|
|    NE|    CE|              4.86730769|
|     N|    AM|              4.86640000|
|    NE|    RN|              4.85850000|
|    CO|    MS|              4.85714286|
|    NE|    SE|              4.83850000|
|    CO|    MT|              4.82442857|
|     N|    RO|              4.82300000|
|     N|    AP|              4.80850000|
|     N|    PA|              4.80225000|
|    CO|    DF|              4.79400000|
|     N|    TO|              4.77160000|
|    NE|    BA|              4.77043333|
|    NE|    PI|              4.76750000|
|    NE|    PB|              4.72750000|
|    SE|    RJ|              4.72171875|
|     S|    SC|              4.72171429|
+------+------+------------------------+
only showing top

### 4. Qual o ágio médio entre os preços de revenda e distribuição de combustíveis por mês, produto, estado?

Combinamos as duas tabelas para calcular a razão preço de revenda / preço de distribuição.

Exemplo de execução para o preço da gasolina no último mês da série.

In [23]:
df_result = (
    spark.table("precos_anp.media_venda_combustiveis_automotivos").alias("v")
    .join(
        spark.table("precos_anp.media_distribuicao_combustiveis_automotivos").alias("d"),
        (F.col("v.mes") == F.col("d.mes")) & 
        (F.col("v.regiao") == F.col("d.regiao")) & 
        (F.col("v.estado") == F.col("d.estado")) & 
        (F.col("v.produto") == F.col("d.produto"))
    )
    .filter((F.col("v.mes") == '2023-06') & (F.col("v.produto") == 'GASOLINA'))
    .select(
        F.col("v.mes"), 
        F.col("v.regiao"), 
        F.col("v.estado"), 
        F.col("v.produto"),
        (F.col("v.valor_venda_medio") / F.col("d.valor_distribuicao_medio")).alias("agio_preco_medio")
    )
    .orderBy("agio_preco_medio", ascending=False)
)

df_result.show()

+-------+------+------+--------+--------------------+
|    mes|regiao|estado| produto|    agio_preco_medio|
+-------+------+------+--------+--------------------+
|2023-06|     N|    AM|GASOLINA|1.273292074223245...|
|2023-06|     N|    RO|GASOLINA|1.225693342318059...|
|2023-06|     N|    AC|GASOLINA|1.212454611409262...|
|2023-06|     N|    TO|GASOLINA|1.176442599966468...|
|2023-06|    NE|    RN|GASOLINA|1.175197795615930...|
|2023-06|     S|    SC|GASOLINA|1.175124045889697...|
|2023-06|     S|    PR|GASOLINA|1.172031398766201...|
|2023-06|     N|    RR|GASOLINA|1.170148378158109...|
|2023-06|    NE|    BA|GASOLINA|1.168343692165172...|
|2023-06|     S|    RS|GASOLINA|1.154487663521395...|
|2023-06|    SE|    SP|GASOLINA|1.154090416722058...|
|2023-06|    CO|    GO|GASOLINA|1.151659889770844...|
|2023-06|    NE|    PE|GASOLINA|1.149338792825854...|
|2023-06|    SE|    RJ|GASOLINA|1.147539874118931...|
|2023-06|    SE|    ES|GASOLINA|1.145451396235699...|
|2023-06|     N|    PA|GASOL

### 5. Em quais estados é mais vantajoso abastecer o carro com etanol?

Diz-se que se o preço do etanol está menor do que 70% do preço da gasolina, é mais vantajoso abastecer com etanol.

Podemos então consultar a razão entre os preços médios de revenda do etanol e da gasolina.

In [27]:
df_gasolina = (
    spark.table("precos_anp.media_venda_combustiveis_automotivos")
    .filter(F.col("produto") == "GASOLINA")
)

df_etanol = (
    spark.table("precos_anp.media_venda_combustiveis_automotivos")
    .filter(F.col("produto") == "ETANOL")
)

df_result = (
    df_gasolina.alias("g")
    .join(
        df_etanol.alias("e"),
        (F.col("g.mes") == F.col("e.mes")) & 
        (F.col("g.regiao") == F.col("e.regiao")) & 
        (F.col("g.estado") == F.col("e.estado"))
    )
    .filter(F.col("g.mes") == '2023-06')
    .select(
        F.col("g.mes"), 
        F.col("g.regiao"), 
        F.col("g.estado"),
        (F.col("e.valor_venda_medio") / F.col("g.valor_venda_medio")).alias("razao_etanol_gasolina")
    )
    .orderBy("razao_etanol_gasolina", ascending=False)
)

df_result.show()

+-------+------+------+---------------------+
|    mes|regiao|estado|razao_etanol_gasolina|
+-------+------+------+---------------------+
|2023-06|     N|    AP| 1.030704033246293...|
|2023-06|     S|    RS| 0.885060910470631...|
|2023-06|     N|    RR| 0.881929684576812...|
|2023-06|    NE|    MA| 0.879345178385414...|
|2023-06|     N|    PA| 0.848628536797618...|
|2023-06|    NE|    CE| 0.837197902972522...|
|2023-06|     S|    SC| 0.833240311069458...|
|2023-06|     N|    RO| 0.832856064292199...|
|2023-06|    NE|    PI| 0.817529434254434...|
|2023-06|    NE|    RN| 0.816962007393143...|
|2023-06|    NE|    AL| 0.815443875201121...|
|2023-06|    NE|    SE| 0.811826172936658...|
|2023-06|     N|    TO| 0.807027796749704...|
|2023-06|    SE|    RJ| 0.799463174888690...|
|2023-06|    SE|    ES| 0.792597311592010...|
|2023-06|    NE|    PE| 0.791263065929456...|
|2023-06|    NE|    BA| 0.786031473487201...|
|2023-06|     N|    AC| 0.783112344487495...|
|2023-06|    NE|    PB| 0.78279891